<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Install-gentle" data-toc-modified-id="Install-gentle-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Install gentle</a></span></li><li><span><a href="#Test-aligning-transcript-with-local-video" data-toc-modified-id="Test-aligning-transcript-with-local-video-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test aligning transcript with local video</a></span></li><li><span><a href="#Test-aligning-transcript-with-Esper-to-retrieve-audio-(not-done)" data-toc-modified-id="Test-aligning-transcript-with-Esper-to-retrieve-audio-(not-done)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Test aligning transcript with Esper to retrieve audio (not done)</a></span></li><li><span><a href="#Test-aligning-transcript-using-scanner-pipeline" data-toc-modified-id="Test-aligning-transcript-using-scanner-pipeline-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test aligning transcript using scanner pipeline</a></span></li></ul></div>

# Install gentle 

1.
git clone https://github.com/scanner-research/gentle

2.
bash ./install.sh (takes ~30min)   

# Test aligning transcript with local video

In [ ]:
# set test video list
video_list = ['CNN_20091231_160000_CNN_Newsroom']

# import pickle
# video_list = pickle.load(open('../app/data/tvnews_std_sample.pkl', 'rb'))['sample_100']

In [ ]:
from scannertools.transcript_alignment import TranscriptAligner
import os
import pickle

res_stats = {}
for video_name in video_list:
    print(video_name)
    # download video
    video_path = os.path.join('../app/data/videos/', video_name+'.mp4')
    if not os.path.exists(video_path):
        gs_path = os.path.join('gs://esper/tvnews/videos/', video_name+'.mp4')
        cmd = 'gsutil cp ' + gs_path + ' ' + '../app/data/videos/'
        os.system(cmd)
    print('Downloading video done')
    
    # run alignment
    aligner = TranscriptAligner(seg_length=60, max_misalign=10, num_thread=64, 
                            media_path=video_path,
                            transcript_path=os.path.join('../app/data/subs10/', video_name),
                            align_dir='../app/result/aligned_transcript/')
    res = aligner.run_all()    
    res_stats[video_name] = res
    pickle.dump(res_stats, open('../app/result/test_align_1.pkl', 'wb'))

# Test aligning transcript using scanner pipeline

In [ ]:
from scannertools import audio, transcript_alignment
import scannerpy
import os
from query.models import Video
SEG_LENGTH = 60

In [ ]:
# set test video list
video_list = ['CNNW_20161229_060000_CNN_Newsroom_Live']
# video_list = ['FOXNEWS_20110326_050000_Greta_Van_Susteren']

# import pickle
# video_list = pickle.load(open('../app/data/tvnews_std_sample.pkl', 'rb'))['sample_100']

In [ ]:
# load videos from database
videos = [Video.objects.filter(path__contains=video_name)[0] for video_name in video_list]

In [ ]:
# get audio length 
import tempfile
import re
def get_audio_length(video):
    url = video.url()
    log = tempfile.NamedTemporaryFile(suffix='.txt').name
    cmd = 'ffprobe -show_streams -i ' + \
        '\"' + url + '\"' + ' > ' + log
    os.system(cmd)
    format_str = open(log, 'r').read()
    return float(re.findall(r'\nduration=(.*)', format_str)[1])

audio_length = [get_audio_length(video) for video in videos]
audio_length

In [ ]:
# load audios from videos
audios = [audio.AudioSource(video.for_scannertools(), frame_size=SEG_LENGTH) for video in videos]

In [ ]:
# set up transcripts 
captions = [audio.CaptionSource('tvnews/subs10/'+video_name, max_time=length , window_size=SEG_LENGTH) 
                for video_name, length in zip(video_list, audio_length)]

In [ ]:
db = scannerpy.Database()
result = transcript_alignment.align_transcript(db, video_list=video_list, audio=audios, caption=captions, 
                                               cache=False, 
                                               align_dir='/app/app/result/aligned_transcript',
                                               res_path='/app/app/result/test_align_1.pkl')

In [ ]:
result[0]._column._table.profiler().write_trace('/app/app/result/test_align_1.trace')